
## Analytics Specializations & Applications - Week 4

# Text Analytics 1 - Preparatory Exercises
----------

There is a growing specialization in business analytics that is concerned with Text Analysis and natural language processing (NLP), as it allows businesses to interrogate the mass of unstructured data that exists about their customers, products and interactions - data that far outweighs the "structured" datasets we have considered up to now.

Analysing such data is non-trivial. Computers are good at understanding rigidly-structured data with limited variety. However, when we deal with unstructured free-form text, things begin to get difficult. Developing NLP applications for business is therefore challenging because computers have a hard time understanding underlying concepts.

Even within a single language there are many subtle variations to the way in which we communicate to consider - tone, sentiment, irony, dialects, context, slang, and so on. In order to solve this problem, NLP analysis has moved from simple statistical analysis, to the use of predictive models to underpin the way the function - Multinational businesses now use such techniques extensively to  deliver relevant results to their workflows. 

Some of the most common applications of NLP include search engines, sentiment analysis, topic modeling, part-of-speech tagging, entity recognition, and so on. The goal of NLP is to develop a set of algorithms so that a company can interact with text written in plain English to use some of the outputs in their businesses processes. We are going to explore how to do this below, in the context of customer reviews for products.

Before we get into the scenario proper, let's just get some of the basic's under our belt...

### Task 4a: Setting up your tools 
At the heart of this case study we will be taking advantage a Python tool that will do a lot of the hard work for us, called the **Natural Language Toolkit (NLTK)**. Make sure that you install this before you proceed.  You can find the installation steps at http://www.nltk.org/install.html for when you do this on your own machines, but on your N/LAB laptop simply type the following lines:

**sudo pip3 install nltk**

** python3 -m nltk.downloader all**

<span style="font-weight:bold; color:green;">&rarr; Check that it has worked by running the following block:<span/>

In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
nltk.download('averaged_perceptron_tagger')

NameError: name 'nltk' is not defined

In [ ]:
import nltk
nltk.download('all')

print(nltk.__version__)

As of the time of writing, the latest version of NLTK is 3.2.4 so that's what you shoudl see above. NLTK also brings with it a host of text corpora that we can use to test on, so let's check that you have access to these by looking at the "brown" corpus

This was the first million-word elecontric corpus of English created from 500 sources in 1961 (at Brown University). 

In [ ]:
from nltk.corpus import brown

#-- list the categories of document available in this corpus
print(brown.categories())

#-- list the words used in the news (from the Chicago Tribune)
print(brown.words(categories="news"))


The brown corpus is split into several categories, but it is itself just one of the many sample text corpuses that you can be accessed and played with directly in NLTK. One that might be relevent to us at some point are the Reuters and Webtext datasets (and we will use a corpus of classic literature texts at the end of these preparatory exercies). If you're interested at what else comes along with NLTK, then take a look at those available here:

<a href="http://www.nltk.org/book/ch02.html">www.nltk.org/book/ch02.html</a>

### Task 4b: Basic text statistics 

Tokenization is the process of dividing text into a set of meaningful pieces. These pieces are called
tokens. For example, we can divide a chunk of text into words, or we can divide it into sentences - or letters and punctuation if we so desired. It all depends on the task at hand. Predominantly, however, we will want to turn text into lists of words. Let's take a look at how to do this.

<span style="font-weight:bold; color:green;">&rarr; Run and examine the following code:<span/>

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
text = """Are you curious about the role of text analytics?     
    Are you ready to see how we can use the unstructured text available throughout the web?
    Well it's 2-o'clock so the time is nigh to do some analysis. """

tokens = nltk.sent_tokenize(text)
print(tokens)

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
tokens = nltk.word_tokenize(text)
print(tokens)

In [ ]:
tokens = nltk.wordpunct_tokenize(text)
print(tokens)

In [ ]:
tokens = nltk.RegexpTokenizer(r'\w+').tokenize(text)
print(tokens)

What was the difference between these various approaches?

Once you have understood what they are each individaully doing, let's move on to examining how our words are distributed...

In [ ]:
#--- see a distribution of the words in the text
fdist = nltk.FreqDist(tokens)
display(fdist)

#-- list the 3 most common words in the text
print(fdist.most_common(10))


Note how the most common words are not particularly useful semantically - in fact they tend to look like stop words. 

See if you can create a bar plot of them, to make it easier to visualize:

In [ ]:
#-- and get a sense of the word counts visually
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (20.0, 9.0)

plt.bar(fdist.keys(), fdist.values())
#-- if the above does not work on your version of python use the following:
#plt.bar(range(len(fdist)), fdist.values(), tick_label = fdist.keys())


In [ ]:
#-- another neat option is as follows using pandas
import pandas 

#-- convert the counter object into a pandas dataframe
df = pandas.DataFrame.from_dict(fdist, orient="index")

#-- use the dataframes built in bar chart function
df.plot.bar()

In [ ]:
#-- or you might prefer a pie chart given we have only a limited number of words...
df.plot.pie(subplots=True, figsize=(10,10))

### Task 4c: The famous "Bag of Words" approach
Instead of just counting words, we can actually use this simple approach to represent a document. This is called a "bag of words" approach, as the document is described with no regards to the ordering of the words, but simply their counts. This turns out to still be a very powerful and flexible approach, allowing us to compare the similarity between two pieces of text.

To do this we use a "Vector Space" representation of a document. First we consider every word that can appear in a corpus (i.e. across all the documents we have) as a feature in it's own right. We then simply assign the value of that feature for any individual doucment, as the number of times that word appears. This means if we have M words in our "dictionary" of words that appear across the corpus, each document can be represented by m features, and we are back in our tradiaional home of seeing a document as a point in an m-dimensional space.

By way of example consider a universe where the are only 3 possible words: "Apple", "Orange" and "Banana". Every sentence of document is made out of them. In this case every document can be represented in a 3 dimensional space. Each word is an axis of that space:

Apple  --> [1,0,0] <br/>
Banana --> [0,1,0] <br/>
Orange --> [0,1,0]

Then a sentence or a document is simply a linear combination of these vectors, where the number of times that word appears is the coefficient along that dimension. For example consider:

dM = "Banana Apple Orange"                       --> [1, 1, 1] <br/>
d1 = "Banana Apple Banana Banana Banana Apple"   --> [2, 4, 0] <br/>
d2 = "Banana Orange Banana Banana Orange Banana" --> [0, 4, 2] <br/>
d3 = "Apple Orange Orange Apple"                 --> [2, 0, 2] <br/>
d4 = "Apple Banana Apple Banana"                 --> [2, 2, 0] <br/>
d5 = "Banana Apple Banana Banana Orange Banana"  --> [1, 4, 1]

If we plotted all of these individual datapoints in a space we roughly get the following:

<img src="vsm.png" style="width:65%; clear:both; margin-bottom:-20px" align=left>
<br style="clear:both;"/>

<br/>
Now the similarity of the documents to each other, or a new "query" document can be assessed. This time sklearn helps nltk along (perhaps this is not surprising given we've converted text into a "feature space") by providing a really useful object called **CountVectorizer()** that we can fit to our data, illicits and counts the unique words in the documents, and converts them to vector form all in one foul swoop.

<span style="font-weight:bold; color:green;">&rarr; Run and examine the following code:<span/>

In [ ]:
corpus = [
    "Banana Apple Orange",
    "Banana Apple Banana Banana Banana Apple",
    "Banana Orange Banana Banana Orange Banana",
    "Apple Orange Orange Apple",
    "Apple Banana Apple Banana",
    "Banana Apple Banana Banana Orange Banana"
]

#-- import and create a vectorizer object
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

#-- set it running with our data
cv.fit(corpus)

#-- extract the names of the unique tokens (features) that it found
features = cv.get_feature_names()
print(features)

#-- and finally use those features to create a vector (datapoint) for each document
vec = cv.transform(corpus).toarray()
print(vec)      

Do those vectors match the ones in the picture? Well it's not easy to tell, so why don't we just create our own 3D plot.

<span style="font-weight:bold; color:green;">&rarr; Run and examine the following code:<span/>

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot('111', projection='3d' )
ax.set_xlabel("APPLE")
ax.set_ylabel("BANANA")
ax.set_zlabel("ORANGE")

#-- set the angle we'll view the 3D plot from:
ax.view_init(20, 20)

#-- and for each point (linked to its text) plot a vector from the origin 
for [x,y,z], c in zip(vec, corpus):
    ax.plot([0,x],[0,y],[0,z])
    ax.text(x, y, z, c)

You'll probably note that the results look extremely similar to our original estimated diagram. Excellent. But what about that "query" document?

Well, we have turned the problem of finding similarity between documents into one you might recognize - finding the distance between two datapoints in a feature space, something we have done before (recall that models such as K-nearest neighbour classifiers and hierarchical clustering in the past required us to calculate distances between points).

In the past we've used Euclidian distances to measure the distances between points. Let's quickly note how to do this here:

In [ ]:
# This is often called a "Document Term Matrix"
from sklearn.metrics.pairwise import euclidean_distances
print(euclidean_distances(vec))

In the case of text documents though, this would not be a good measure to use. Why? Well it is to dependent on the length of the document. We care about semantic content not number of words, so instead a distance measure called cosine similarity. This measure the angle between two document vectors, which inherently describes their semantic similarity, without caring aobut the length of the vector (i.e. document) itself. Observe how it produces different results:

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(vec))

There are several issues with this approach when you consider the entire English dictionary. For example:


In fact, if that document had a label we'd have been able to categorize our most recent document, and we'd have built a document classification algorithm. This is just what a spam filter does, based on documents it knows as "spam".

<span style="font-weight:bold; color:green;">&rarr;Use python to determine which document is the  query document "Banana Orange Banana" most similar to?
<span/>

In [ ]:
#-- to be used by the vectorizer, even our query has to be a list (of one)
q = ["Banana Orange Banana"]
q_vec = cv.transform(q)
print(q_vec.toarray())

#-- calculate the cosine similarity with the other documents encoded in vec
print(cosine_similarity(q_vec, vec))

Which did you match our query to? Remember we want the match with the highest cosine similarity (if we'd used Euclidean distance we'd have wanted the lowest distance score, not the hightest!)

Hopefully you got document 2 (D2) as it was the same semantic content, just half as long! If you didn't go back and check your working before moving on.


### Challenges for real world text analytics 
Okay, so we have made excellent headway wiht our toy text analytics examples. We have a way of encoding and analysing documents that allows us to also summarize them and find similarity between them. 

But we have, admittedly, only used toy datasets, and as ever, when we user larger, real world dataset issues arise:

> 1. In any language there are many words that are very common but don't offer much semantic information ("e.g. "the", "and", "but", etc.). We could do with stopping these words from being incorporated into our analyses<br/><br/>
> 2. Words that have the same root (and hence the same meaning) are being treated differently (e.g. The words "purchase", "puchasing" and "purchased" have essentially the same meaning but look completely different to the analysis)
> 3. In any given corpus there will be words that appear in a lot of documents. (e.g. in customer reviews of vehicles, words like "car", "drive", etc. are not paricularly useful in dilienating differences in reviews, as they will appear in every review!). We should deal with this.

We're going to consider each of these core issues in turn, before we apply everything to a real scenario. The last issue is actually potentially the most important, so let's deal with that first.

### Task 4d: Accounting for Term Frequency in the corpus

Term Frequency Inverse Document Frequency (TF-IDF) is a very good solution for problems 3 in the above list. Currently we are using direct word/term counts (i.e. Term frequency). The approach we will try now accounts for the fact that some of those terms are in every document and so their importance should be weighted down...

#### IDF: (Inverse document frequency)
The idea here is to penalize the total count of a word in a document by the number of its appearance in all of the documents. The higher this number the less valuable the word is – it contains less information that can identify the document (n.b. In the extreme case, where the word appears in large fraction of the documents, usually it is even better to completely eliminate the count - we will come to this in the stopwords section).

A good heuristic for a words importance is **log((1+m)/k)** where m is the number of the documents in the whole corpus and k is the number of those that the word appears in. For reference, this importance heuristic can be visualized as follows:

<img src="IDF.png" style="width:75%; clear:both; margin-bottom:-20px" align=left>
<br style="clear:both;"/>

#### TF: (Term frequency)
Another problem is how many times shall we count a word in a document? So far we have been counting terms as many times as they appear, but another extreme would be to just count them once and ignore any other appearance of the word. In many search engines, something in between is used. However, in general for business cases, we simply use the amount of times a word occurs in the document - it's term frequency ("TF").

#### TF-IDF: (Term frequency Inverse Document Frequency)
For a final score for a term's importance we no longer just use its term frequency, but divide by the terms IDF. 

Luckily we don't have to do any of this manually - NLTK wraps it all up for us by a new version of the count vectorizer with TF-IDF incorporated in, called (unsurprisingly) the **TfidfVectorizer()**:

In [ ]:
#-- import and create a vectorizer object
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer()

#-- set it running with our data
tv.fit(corpus)
tv_vec = tv.transform(corpus).toarray()
print(tv_vec)    

You'll notice these are slightly different results to before, with each document have slightly different coordinates in the feature space now. However,t the process was identical, and this is the vectorization tool that you should genearally use in all your business cases from now on.


For more details of TF-IDF see <a href="http://www.tfidf.com">www.tfidf.com</a>

### Task 4e: Removing Stop Words
While we have now weighted terms by importance, some words are just pointless to include in analyses, and just need to be eradicated. In text analytics, "stop words" are those words which are filtered out before we do our real analysis due to the fact they do not add any real semantic content but do add to the curse of dimensionality (in fact many stop-words exist in the language more for the purposes of redundancy and grammar than to add meaning).

While "stop words" usually refers to the most common words in a language, there is no single universal list of stop words used by all natural language processing tools - and indeed not all business anlaytics tools even use such a list. 

Any group of words can be chosen as the stop words for a given purpose. For many business contexts (such as search engines) these tend to be common, short function words, such as "the", "is", "at", "which", and on. Occasionally this word removal can generate problems (there is famously a band called "The The" that suffers particularly badly from stop word filtering!).

Let's return to our original text, and examine how we can get rid of stopwords. NLTK provides a list of stopwords that we will use:

In [ ]:
import nltk
nltk.download('stopwords')


In [ ]:
from nltk.corpus import stopwords
remove_words = stopwords.words('english')
print(remove_words)

Now we can use this list to filter our own tokens:

In [ ]:
#-- One method for removing stop words from your token list
filtered_tokens = []
for t in tokens:
    t = t.lower()
    if t not in remove_words:
        filtered_tokens.append(t)

#-- Examine the results
print("ORIGINAL SET OF TOKENS:", tokens)
print("FILTERED FOR STOP WORDS:", filtered_tokens)

Note that we had to set all the text to lowercase to make the matches. However, once achieved and now we don't have stop words littering our text, we will get a lot more accurate and descriptive results.

### Task 4f: Stemming
However, we still have another challenge, even once stop words have been removed - we have words with the same root, and hence the same inherent meaning, being treated completely orthogonally. This is addressed by a process called "stemming" which reduces a word down to it's core "stem".

NLTK has again built in tools to allow us to do this very 

In [ ]:
#-- One method for removing stop words from your token list
stemmer = nltk.PorterStemmer()

for t in tokens:
    print("{} --> {}".format(t, stemmer.stem(t)))

In [ ]:
#-- One method for removing stop words and stemming words in your documents
stemmed_text = ""
for t in tokens:
    t = stemmer.stem(t.lower())
    if t not in remove_words:
        stemmed_text = stemmed_text + t + " "

#-- Examine the results
print("ORIGINAL SET OF TOKENS:", text)
print()
print("FILTERED AND STEMMED:", stemmed_text)


### Task 4g: Bringing it all together...

We shall now take all the tools from above and as I final prepartory exerciese use them to analyse some classic books from English literature, before moving onto our business case looking at customer reviews.

Any questions about what is happening in this code, please ask:

In [ ]:
#-- First we are going to use some of the files in the 'gutenburg' corpus
#-- that is provided as part of the nltk package
files = ['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'carroll-alice.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt' ]


#-- slightly adapted version of our text cleaner from above,
#-- as a function that also removes punctuation 
import string
def clean(input_text):
    result = ""
    for t in input_text:
        t = stemmer.stem(t.lower())
        if t not in remove_words and t not in string.punctuation:
            result = result + " " + t
    return result


#-- import and clean all the words from each book in the file list
book_data = []
for f in files:
    #-- load in the words from the book as tokens
    #-- using nltk's built in functionality
    raw_tokens = nltk.corpus.gutenberg.words(f)
    
    #-- clean the current books text using our function
    cleaned_text = clean(raw_tokens)
    
    #-- preview the results of this cleaning process
    print("cleaning -->", cleaned_text[:50])
    
    #-- and add the cleansed book text back to our overall data
    book_data.append(cleaned_text)

In [ ]:
#-- now we have cleansed data let's turn into into datapoints
#-- in our  feature space
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer()

#-- fit a vectorizer with all of our book data
tv.fit(book_data)

#-- create a vector representation for each book we used
tv_vec = tv.transform(book_data)

#-- and then let's just check how many words/terms we found and used 
n = len(tv.get_feature_names())
print("The classic books are now represented by {} word features".format(n))

Finally let's create a query bit of text, and see which author it is most similar to!

In [ ]:
#-- create a query
q = """
    'But we don't want to among mad people', the MSc class exclaimed.
    'Oh, you can't help that', said James, 'you see, we're all mad here...
     I'm mad. You're mad. All of us are I'm afraid'.
    'Ludicrous', said the class in unison, 'How could you know if we were mad?'.
    'You must be', said James, 'or you wouldn't ever have come here at all'.
"""
q_vec = tv.transform([q])

#-- calculate the cosine similarity with the other documents encoded in vec
similarities = cosine_similarity(q_vec, tv_vec)[0]

#-- list the similarity scores for each book
for a,b in zip(similarities, files):
    print(a, b)

Try some different texts, and see if you can get a tone that sounds like a certain author (Lewis Caroll was probably the easiest one! Shakespeare's prose might be more difficult to mimic) before we move on.

That concludes all the basics we need for our scenario of analysing sentiment in customer reviews. Make sure you understand the concepts of this workflow, in readiness for cutting and pasting structure into your actual case studies. And remember - this is not syntax you need to permanently memorize, you just need to now how to find, use and adapt it.

<span style="font-weight:bold; color:green;">&rarr; Please now move on to the "Week 4 - Case Study Exercises" notebook:<span/>